In [3]:

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
import pyspark
import pandas as pd
from pyspark.sql.utils import AnalysisException


In [4]:
ss_name = 'Ejecucion algoritmo IVA Credito Remanente'
wg_conn = "spark.kerberos.access.hadoopFileSystems"
db_conn = "abfs://data@datalakesii.dfs.core.windows.net/"

spark = SparkSession.builder \
      .appName(f"Ejecucion algoritmo {ss_name}")  \
      .config(wg_conn, db_conn) \
      .config("spark.executor.memory", "6g") \
      .config("spark.driver.memory", "12g")\
      .config("spark.executor.cores", "4") \
      .config("spark.executor.instances", "5") \
      .config("spark.driver.maxResultSize", "12g") \
      .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

spark.conf.set("spark.sql.parquet.enableVectorizedReader","false")
spark.conf.set("spark.sql.parquet.int96RebaseModeInRead", "CORRECTED")
spark.conf.set("spark.sql.parquet.int96RebaseModeInWrite", "CORRECTED")
spark.conf.set("spark.sql.parquet.datetimeRebaseModeInRead", "CORRECTED")
spark.conf.set("spark.sql.parquet.datetimeRebaseModeInWrite", "CORRECTED")



Setting spark.hadoop.yarn.resourcemanager.principal to hvega.externo


In [4]:
df_spark=spark.read.parquet("abfs://data@datalakesii.dfs.core.windows.net/DatosOrigen/lr-629/riesgo_fraude/remanente_contaminado/remanente_hist_updated")


In [8]:
df_spark=spark.sql('select * from dw.dw_trn_f29_e')

In [9]:
# Número de columnas
num_columns = len(df_spark.columns)
print(f"Número de columnas: {num_columns}")

# Número de filas
num_rows = df_spark.count()
print(f"Número de filas: {num_rows}")

Número de columnas: 365


Número de filas: 113122399


In [ ]:
# Seleccionar los valores únicos de la columna "columna_ejemplo"
valores_unicos_ordenados = df_spark.select("dcv_ptributario").distinct().orderBy(col("dcv_ptributario").desc())


# Mostrar los valores únicos
valores_unicos_ordenados.show()

In [ ]:
spark.stop()

In [ ]:
import pyspark.sql.functions as F

# Convertir columnas de fecha y hora a cadenas en un solo paso
def convert_date_time_columns(df):
    date_time_cols = [c for c in df.columns if 'timestamp' in df.schema[c].dataType.simpleString() or 'date' in df.schema[c].dataType.simpleString()]
    for col_name in date_time_cols:
        df = df.withColumn(col_name, F.date_format(F.col(col_name), "yyyy-MM-dd HH:mm:ss"))
    return df

# Aplicar la conversión
df_spark_converted = convert_date_time_columns(df_spark)

In [ ]:
# Guardar el DataFrame de Spark como un archivo CSV
df_spark_converted.write \
    .format("csv") \
    .option("header", "true") \
    .mode("overwrite") \
    .save("/home/cdsw/data/remanente_ejemplo.csv")

In [ ]:
spark.stop()

In [ ]:
from datetime import datetime, timedelta
def generate_periods_from_today_as_integers(num_years):
    # Obtener la fecha actual
    today = datetime.today()
    
    # Inicializar la lista de periodos
    periods = []
    
    # Fecha de inicio es el primer día del mes actual
    start_date = today.replace(day=1)
    
    # Calcular la fecha final (hace num_years años)
    end_date = start_date - timedelta(days=num_years * 365)
    
    # Generar periodos en formato YYYYMM
    current_date = start_date
    while current_date > end_date:
        # Convertir el periodo a entero y añadir a la lista al inicio
        period_as_int = int(current_date.strftime("%Y%m"))
        periods.insert(0, period_as_int)  # Inserta al principio de la lista
        # Mover al mes anterior
        next_month = current_date.month - 1 if current_date.month > 1 else 12
        next_year = current_date.year - 1 if current_date.month == 1 else current_date.year
        current_date = current_date.replace(year=next_year, month=next_month, day=1)
    
    # Omitir el último periodo (mes en el cual me detengo)
    if periods:
        periods.pop(0)
    
    return periods
# Ejemplo de uso
num_years = 3
periods_list = generate_periods_from_today_as_integers(num_years)
print(periods_list)

In [ ]:
from datetime import datetime, timedelta

def generate_periods_from_today_as_integers(num_years):
    # Obtener la fecha actual
    today = datetime.today()
    
    # Inicializar la lista de periodos
    periods = []
    
    # Fecha de inicio es el primer día del mes actual
    start_date = today.replace(day=1)
    
    # Calcular la fecha final (hace num_years años), no incluyendo el mes actual
    end_date = start_date - timedelta(days=num_years * 365)
    
    # Ajustar end_date para que sea el primer día del mes del periodo que queremos excluir
    end_date = end_date.replace(day=1)
    
    # Generar periodos en formato YYYYMM
    current_date = start_date
    while current_date > end_date:
        # Convertir el periodo a entero y añadir a la lista al inicio
        period_as_int = int(current_date.strftime("%Y%m"))
        periods.insert(0, period_as_int)  # Inserta al principio de la lista
        # Mover al mes anterior
        next_month = current_date.month - 1 if current_date.month > 1 else 12
        next_year = current_date.year - 1 if current_date.month == 1 else current_date.year
        current_date = current_date.replace(year=next_year, month=next_month, day=1)
    
    # El último periodo será el mes anterior al actual
    if periods:
        periods.pop(0)
    
    return periods

# Ejemplo de uso
num_years = 3
periods_list = generate_periods_from_today_as_integers(num_years)
print(periods_list)


In [ ]:
from datetime import datetime, timedelta

def generate_periods_from_today_as_integers(num_years):
    # Obtener la fecha actual
    today = datetime.today()
    
    # Inicializar la lista de periodos
    periods = []
    
    # Fecha de inicio es el primer día del mes actual
    start_date = today.replace(day=1)
    
    # Calcular la fecha final (hace num_years años)
    end_date = start_date - timedelta(days=num_years * 365)
    
    # Ajustar end_date para que sea el primer día del mes del periodo que queremos excluir
    end_date = end_date.replace(day=1)
    
    # Generar periodos en formato YYYYMM
    current_date = start_date
    while current_date > end_date:
        # Convertir el periodo a entero y añadir a la lista al inicio
        period_as_int = int(current_date.strftime("%Y%m"))
        periods.insert(0, period_as_int)  # Inserta al principio de la lista
        # Mover al mes anterior
        next_month = current_date.month - 1 if current_date.month > 1 else 12
        next_year = current_date.year - 1 if current_date.month == 1 else current_date.year
        current_date = current_date.replace(year=next_year, month=next_month, day=1)
    
    # Eliminar el último valor de la lista si no está vacío
    if periods:
        periods.pop()  # Elimina el último elemento
    
    return periods

# Ejemplo de uso
num_years = 3
periods_list = generate_periods_from_today_as_integers(num_years)
print(periods_list)


In [11]:
a=spark.read.parquet("abfs://data@datalakesii.dfs.core.windows.net/DatosOrigen/lr-629/riesgo_fraude/remanente_contaminado/f29_unico")
a.registerTempTable('f29')

StructType(List(StructField(max_folio,StringType,true),StructField(cont_rut,StringType,true),StructField(periodo,IntegerType,true)))

In [13]:
spark.sql('select distinct(f29_agno_mes_tributario_vo) from dw.dw_trn_f29_e order by f29_agno_mes_tributario_vo desc').show()

Hive Session ID = f05b230c-ded8-407d-9572-d5d6b6aff69e


+--------------------------+
|f29_agno_mes_tributario_vo|
+--------------------------+
|                    202307|
|                    202306|
|                    202305|
|                    202304|
|                    202303|
|                    202302|
|                    202301|
|                    202212|
|                    202211|
|                    202210|
|                    202209|
|                    202208|
|                    202207|
|                    202206|
|                    202205|
|                    202204|
|                    202203|
|                    202202|
|                    202201|
|                    202112|
+--------------------------+
only showing top 20 rows



In [15]:
spark.sql('select distinct(dcv_ptributario) from dwbgdata.dcv_generic_det_consolidado_sas order by dcv_ptributario desc').show()

+---------------+
|dcv_ptributario|
+---------------+
|         202402|
|         202401|
|         202312|
|         202311|
|         202310|
|         202309|
|         202308|
|         202307|
|         202306|
|         202305|
|         202304|
|         202303|
|         202302|
|         202301|
|         202212|
|         202211|
|         202210|
|         202209|
|         202208|
|         202207|
+---------------+
only showing top 20 rows

